In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!unzip '/content/drive/My Drive/523_pipeline/output_image_64000_addition.zip' -d '/content/drive/My Drive/523_pipeline/output_image_64000_addition'
#!unzip '/content/drive/My Drive/523_pipeline/output_image_latest(1).zip' -d '/content/drive/My Drive/523_pipeline/output_image_latest(1)'

Archive:  /content/drive/My Drive/523_pipeline/output_image_64000_addition.zip
   creating: /content/drive/My Drive/523_pipeline/output_image_64000_addition/output_image_64000_addition/character/
  inflating: /content/drive/My Drive/523_pipeline/output_image_64000_addition/output_image_64000_addition/character/character_progress_0.png  
  inflating: /content/drive/My Drive/523_pipeline/output_image_64000_addition/output_image_64000_addition/character/character_progress_1.png  
  inflating: /content/drive/My Drive/523_pipeline/output_image_64000_addition/output_image_64000_addition/character/character_progress_10.png  
  inflating: /content/drive/My Drive/523_pipeline/output_image_64000_addition/output_image_64000_addition/character/character_progress_100.png  
  inflating: /content/drive/My Drive/523_pipeline/output_image_64000_addition/output_image_64000_addition/character/character_progress_101.png  
  inflating: /content/drive/My Drive/523_pipeline/output_image_64000_addition/output

In [4]:
generated_dir = '/content/drive/My Drive/523_pipeline/output_image_64000_addition/output_image_64000_addition/generated'
ground_truth_dir = '/content/drive/My Drive/523_pipeline/output_image_64000_addition/output_image_64000_addition/ground_truth'

In [9]:
len(ground_truth_dir)

105

In [5]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import os
import numpy as np

# Load a pre-trained ResNet model
model = models.resnet50(pretrained=True)
model.eval()  # Set the model to evaluation mode

# Define a transform to preprocess the image
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to load an image and transform it
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

# Function to extract features using the PyTorch model
def extract_features(input_image, model):
    with torch.no_grad():
        features = model(input_image)
    return features

# Initialize the list to store the highest similarity score for each ground truth image
scores = []

# Loop through each ground truth image
for i in range(302):
    gt_img_path = os.path.join(ground_truth_dir, f'ground_truth_progress_{i}.png')
    gt_image_tensor = load_image(gt_img_path)
    gt_features = extract_features(gt_image_tensor, model)

    max_similarity = 0  # Initialize the maximum similarity score

    # Loop through the corresponding generated images
    for k in range(3):
        gen_img_path = os.path.join(generated_dir, f'generated_progress_{i}_{k}.png')
        gen_image_tensor = load_image(gen_img_path)
        gen_features = extract_features(gen_image_tensor, model)

        # Calculate cosine similarity
        similarity = torch.nn.functional.cosine_similarity(gen_features, gt_features, dim=1).item()
        if similarity > max_similarity:
            max_similarity = similarity

    scores.append(max_similarity)  # Append the highest score

# scores now contains the highest similarity scores for each ground truth image


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 150MB/s]


In [6]:
scores

[0.8236818313598633,
 0.8407078385353088,
 0.7808247804641724,
 0.8463441133499146,
 0.8710339069366455,
 0.7909162044525146,
 0.7994627356529236,
 0.7450000643730164,
 0.7935184836387634,
 0.8281355500221252,
 0.8570970892906189,
 0.8293107748031616,
 0.7292231917381287,
 0.812808632850647,
 0.7502418756484985,
 0.7806127071380615,
 0.6584397554397583,
 0.876710057258606,
 0.7859848737716675,
 0.8626428842544556,
 0.7201645374298096,
 0.7832000255584717,
 0.7886682748794556,
 0.7778646945953369,
 0.6660666465759277,
 0.925709068775177,
 0.814228892326355,
 0.7750160694122314,
 0.7889025211334229,
 0.7893412113189697,
 0.8368927240371704,
 0.782615065574646,
 0.8177255392074585,
 0.8536149859428406,
 0.7816869616508484,
 0.8221345543861389,
 0.8576958775520325,
 0.8395128846168518,
 0.7969583868980408,
 0.7730417847633362,
 0.8306101560592651,
 0.8082952499389648,
 0.7822380065917969,
 0.7833178043365479,
 0.8215789198875427,
 0.7769985795021057,
 0.759905219078064,
 0.9068372249603271

In [7]:
len(scores)

302

In [10]:
average_score = sum(scores) / len(scores)
average_score

0.796188032390266

In [11]:
text_dir = '/content/drive/My Drive/523_pipeline/output_image_64000_addition/output_image_64000_addition/text'

In [12]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os

# Initialize CLIP model and processor
model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')

def calculate_text_image_correlation(image_path, text):
    image = Image.open(image_path).convert('RGB').resize((256, 256))
    inputs = processor(text=[text], images=image, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    similarity = torch.nn.functional.cosine_similarity(outputs.text_embeds, outputs.image_embeds, dim=1)
    return similarity.item()

# List to store the highest similarity score for each ground truth image
text_scores = []

# Loop through each text file (302 total)
for i in range(302):
    text_path = os.path.join(text_dir, f'text_progress_{i}.txt')
    with open(text_path, 'r') as file:
        text = file.read()

    max_similarity = 0  # Initialize the maximum similarity score for this ground truth and its generated images

    # Loop through the corresponding generated images
    for k in range(3):
        gen_img_path = os.path.join(generated_dir, f'generated_progress_{i}_{k}.png')
        similarity = calculate_text_image_correlation(gen_img_path, text)
        if similarity > max_similarity:
            max_similarity = similarity

    text_scores.append(max_similarity)  # Append the highest score for this set of generated images

# Now 'scores' contains the highest similarity scores for each ground truth and its corresponding text


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [13]:
text_scores

[0.25846755504608154,
 0.31635960936546326,
 0.32352903485298157,
 0.32195088267326355,
 0.3321455717086792,
 0.3169873356819153,
 0.3494315445423126,
 0.3067094385623932,
 0.3541102707386017,
 0.3233985900878906,
 0.3331046998500824,
 0.3169713616371155,
 0.3281610906124115,
 0.37657272815704346,
 0.312227338552475,
 0.27049365639686584,
 0.3181266486644745,
 0.3152545988559723,
 0.3226982355117798,
 0.3206990659236908,
 0.3401225805282593,
 0.31104782223701477,
 0.3390718102455139,
 0.3092655539512634,
 0.27476251125335693,
 0.3367534279823303,
 0.3747515380382538,
 0.2956891357898712,
 0.24804171919822693,
 0.2970593571662903,
 0.3271588981151581,
 0.32482415437698364,
 0.3427577614784241,
 0.3192669153213501,
 0.2810518145561218,
 0.33943971991539,
 0.26509779691696167,
 0.31799742579460144,
 0.3460429906845093,
 0.36751407384872437,
 0.3939655125141144,
 0.3167596757411957,
 0.37139931321144104,
 0.31791573762893677,
 0.3308066427707672,
 0.3417515158653259,
 0.33583101630210876,


In [18]:
average_text_score = sum(text_scores) / len(text_scores)
average_text_score

0.33395315568573425

In [15]:
len(text_scores)

302

In [16]:
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch
import os

# Initialize the InceptionV3 model
model = models.inception_v3(pretrained=True)
model.eval()  # Set the model to evaluation mode

# Define image transformation
transform = transforms.Compose([
    transforms.Resize(299),  # Resize the image to fit the input size of the model
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# List of image file paths, assuming all relevant files are PNGs
image_files = sorted([os.path.join(generated_dir, f) for f in os.listdir(generated_dir) if f.endswith('.png')])

# List to store the highest confidence scores for each set of three images
image_quality_score = []

# Process each set of three images
for i in range(0, len(image_files), 3):
    max_confidence = 0  # Initialize the maximum confidence score for this set
    # Process each image in the set of three
    for j in range(3):
        image_path = image_files[i + j]
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0)  # Add a batch dimension

        # Get model prediction and confidence
        with torch.no_grad():
            outputs = model(image)
            _, predicted = outputs.max(1)
            confidence = torch.nn.functional.softmax(outputs, dim=1)[0, predicted].item()

        # Update the maximum confidence found in this triplet
        if confidence > max_confidence:
            max_confidence = confidence

    # Store the highest confidence score from the set
    image_quality_score.append(max_confidence)

# Print or return the list of best confidences
print(image_quality_score)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 141MB/s] 


[0.5149956941604614, 0.1409534364938736, 0.1914616972208023, 0.6064786911010742, 0.4968236982822418, 0.5933094620704651, 0.9881479144096375, 0.594449520111084, 0.5621148347854614, 0.662310004234314, 0.7504756450653076, 0.26632562279701233, 0.6475516557693481, 0.871135413646698, 0.17689117789268494, 0.29910314083099365, 0.6560314297676086, 0.7323307394981384, 0.5153864026069641, 0.518430233001709, 0.08010899275541306, 0.16184136271476746, 0.27561622858047485, 0.5010483264923096, 0.2964296042919159, 0.630279004573822, 0.3521859049797058, 0.5967644453048706, 0.820982813835144, 0.557921826839447, 0.5933716893196106, 0.14065127074718475, 0.7191041111946106, 0.7017387747764587, 0.638276219367981, 0.3469167649745941, 0.15954917669296265, 0.8053048253059387, 0.7329978942871094, 0.16663919389247894, 0.1478392332792282, 0.15200677514076233, 0.3465724289417267, 0.15534226596355438, 0.3198506832122803, 0.3136345446109772, 0.48914244771003723, 0.3610779047012329, 0.8120033144950867, 0.1078985929489

In [17]:
average_text_score = sum(image_quality_score) / len(image_quality_score)
average_text_score

0.4025371321631188